In [2]:
from dataset import AudioMixingDataset
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

In [3]:
d = AudioMixingDataset('/media/apelykh/bottomless-pit/datasets/mixing/MUSDB18HQ/train',
                       chunk_length=5, train_val_test_split=(1.0, 0.0, 0.0),
                       mode='train', seed=123)
print(len(d))

4526


In [28]:
for i in range(len(d)):
    print('CHUNK: {}'.format(i))
    print('---------------')
    sample = d[i]
    print('---------------')

CHUNK: 0
---------------
Song index:  0
[-] Unloading track None
[+] Loading track 0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
Num free chunks:  47
Chunk index:  3
---------------
CHUNK: 1
---------------
Song index:  0
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
Num free chunks:  46
Chunk index:  19
---------------
CHUNK: 2
---------------
Song index:  0
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
Num free chunks:  45
Chunk index:  7
---------------
CHUNK: 3
---------------
Song index:  0
[ 0  1  2  4  5  6  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
Num free chunks:  44
Chunk index:  20
---------------
CHUNK: 4
--------------

---------------
CHUNK: 51
---------------
Song index:  1
[ 0  1  2  3  4  5  6  7  8 11 12 13 14 15 16 17 18 19 21 22 23 24 25 26
 27 28 30 31 32 33 34 35 36 37 38 39 40]
Num free chunks:  37
Chunk index:  21
---------------
CHUNK: 52
---------------
Song index:  1
[ 0  1  2  3  4  5  6  7  8 11 12 13 14 15 16 17 18 19 22 23 24 25 26 27
 28 30 31 32 33 34 35 36 37 38 39 40]
Num free chunks:  36
Chunk index:  16
---------------
CHUNK: 53
---------------
Song index:  1
[ 0  1  2  3  4  5  6  7  8 11 12 13 14 15 17 18 19 22 23 24 25 26 27 28
 30 31 32 33 34 35 36 37 38 39 40]
Num free chunks:  35
Chunk index:  0
---------------
CHUNK: 54
---------------
Song index:  1
[ 1  2  3  4  5  6  7  8 11 12 13 14 15 17 18 19 22 23 24 25 26 27 28 30
 31 32 33 34 35 36 37 38 39 40]
Num free chunks:  34
Chunk index:  28
---------------
CHUNK: 55
---------------
Song index:  1
[ 1  2  3  4  5  6  7  8 11 12 13 14 15 17 18 19 22 23 24 25 26 27 30 31
 32 33 34 35 36 37 38 39 40]
Num free chunks:  33
Chu

---------------
CHUNK: 101
---------------
Song index:  2
[ 0  1  3  6  8  9 10 11 13 14 15 17 18 22 24 25 26 28 30 31 32 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48]
Num free chunks:  36
Chunk index:  45
---------------
CHUNK: 102
---------------
Song index:  2
[ 0  1  3  6  8  9 10 11 13 14 15 17 18 22 24 25 26 28 30 31 32 34 35 36
 37 38 39 40 41 42 43 44 46 47 48]
Num free chunks:  35
Chunk index:  40
---------------
CHUNK: 103
---------------
Song index:  2
[ 0  1  3  6  8  9 10 11 13 14 15 17 18 22 24 25 26 28 30 31 32 34 35 36
 37 38 39 41 42 43 44 46 47 48]
Num free chunks:  34
Chunk index:  43
---------------
CHUNK: 104
---------------
Song index:  2
[ 0  1  3  6  8  9 10 11 13 14 15 17 18 22 24 25 26 28 30 31 32 34 35 36
 37 38 39 41 42 44 46 47 48]
Num free chunks:  33
Chunk index:  30
---------------
CHUNK: 105
---------------
Song index:  2
[ 0  1  3  6  8  9 10 11 13 14 15 17 18 22 24 25 26 28 31 32 34 35 36 37
 38 39 41 42 44 46 47 48]
Num free chunks:  32
Chunk index:

---------------
CHUNK: 152
---------------
Song index:  3
[ 1  2  4  5  6  7  8  9 11 12 14 15 16 17 18 21 23 24 25 26 27 28 29 30
 31 33 36 39 41 42 45]
Num free chunks:  31
Chunk index:  16
---------------
CHUNK: 153
---------------
Song index:  3
[ 1  2  4  5  6  7  8  9 11 12 14 15 17 18 21 23 24 25 26 27 28 29 30 31
 33 36 39 41 42 45]
Num free chunks:  30
Chunk index:  17
---------------
CHUNK: 154
---------------
Song index:  3
[ 1  2  4  5  6  7  8  9 11 12 14 15 18 21 23 24 25 26 27 28 29 30 31 33
 36 39 41 42 45]
Num free chunks:  29
Chunk index:  25
---------------
CHUNK: 155
---------------
Song index:  3
[ 1  2  4  5  6  7  8  9 11 12 14 15 18 21 23 24 26 27 28 29 30 31 33 36
 39 41 42 45]
Num free chunks:  28
Chunk index:  26
---------------
CHUNK: 156
---------------
Song index:  3
[ 1  2  4  5  6  7  8  9 11 12 14 15 18 21 23 24 27 28 29 30 31 33 36 39
 41 42 45]
Num free chunks:  27
Chunk index:  7
---------------
CHUNK: 157
---------------
Song index:  3
[ 1  2  4  5 

---------------
CHUNK: 203
---------------
Song index:  4
[ 0  1  5  7  8  9 13 14 17 22 23 26 27 28 31 34 35 36 37 38 39 40 41 42
 43 44 46 47 48 49]
Num free chunks:  30
Chunk index:  44
---------------
CHUNK: 204
---------------
Song index:  4
[ 0  1  5  7  8  9 13 14 17 22 23 26 27 28 31 34 35 36 37 38 39 40 41 42
 43 46 47 48 49]
Num free chunks:  29
Chunk index:  36
---------------
CHUNK: 205
---------------
Song index:  4
[ 0  1  5  7  8  9 13 14 17 22 23 26 27 28 31 34 35 37 38 39 40 41 42 43
 46 47 48 49]
Num free chunks:  28
Chunk index:  39
---------------
CHUNK: 206
---------------
Song index:  4
[ 0  1  5  7  8  9 13 14 17 22 23 26 27 28 31 34 35 37 38 40 41 42 43 46
 47 48 49]
Num free chunks:  27
Chunk index:  8
---------------
CHUNK: 207
---------------
Song index:  4
[ 0  1  5  7  9 13 14 17 22 23 26 27 28 31 34 35 37 38 40 41 42 43 46 47
 48 49]
Num free chunks:  26
Chunk index:  26
---------------
CHUNK: 208
---------------
Song index:  4
[ 0  1  5  7  9 13 14 17 22 

---------------
CHUNK: 258
---------------
Song index:  6
[ 0  1  2  3  4  6  8 10 11 12 13 15 16 17 18 19 21 22 23 24 25 26 27 28
 31 32 33 34 36 38 39 40 41 42 43 45 46 47 48 49]
Num free chunks:  40
Chunk index:  17
---------------
CHUNK: 259
---------------
Song index:  6
[ 0  1  2  3  4  6  8 10 11 12 13 15 16 18 19 21 22 23 24 25 26 27 28 31
 32 33 34 36 38 39 40 41 42 43 45 46 47 48 49]
Num free chunks:  39
Chunk index:  31
---------------
CHUNK: 260
---------------
Song index:  6
[ 0  1  2  3  4  6  8 10 11 12 13 15 16 18 19 21 22 23 24 25 26 27 28 32
 33 34 36 38 39 40 41 42 43 45 46 47 48 49]
Num free chunks:  38
Chunk index:  12
---------------
CHUNK: 261
---------------
Song index:  6
[ 0  1  2  3  4  6  8 10 11 13 15 16 18 19 21 22 23 24 25 26 27 28 32 33
 34 36 38 39 40 41 42 43 45 46 47 48 49]
Num free chunks:  37
Chunk index:  21
---------------
CHUNK: 262
---------------
Song index:  6
[ 0  1  2  3  4  6  8 10 11 13 15 16 18 19 22 23 24 25 26 27 28 32 33 34
 36 38 39 4

---------------
CHUNK: 308
---------------
Song index:  7
[ 0  3  5  6  7  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 31 32 33 35 36 37 38 39 40 42 43 45 46 47 49]
Num free chunks:  40
Chunk index:  12
---------------
CHUNK: 309
---------------
Song index:  7
[ 0  3  5  6  7  9 10 11 13 14 16 17 18 19 20 21 22 23 24 25 26 27 28 29
 31 32 33 35 36 37 38 39 40 42 43 45 46 47 49]
Num free chunks:  39
Chunk index:  10
---------------
CHUNK: 310
---------------
Song index:  7
[ 0  3  5  6  7  9 11 13 14 16 17 18 19 20 21 22 23 24 25 26 27 28 29 31
 32 33 35 36 37 38 39 40 42 43 45 46 47 49]
Num free chunks:  38
Chunk index:  3
---------------
CHUNK: 311
---------------
Song index:  7
[ 0  5  6  7  9 11 13 14 16 17 18 19 20 21 22 23 24 25 26 27 28 29 31 32
 33 35 36 37 38 39 40 42 43 45 46 47 49]
Num free chunks:  37
Chunk index:  14
---------------
CHUNK: 312
---------------
Song index:  7
[ 0  5  6  7  9 11 13 16 17 18 19 20 21 22 23 24 25 26 27 28 29 31 32 33
 35 36 37 38

---------------
CHUNK: 357
---------------
Song index:  8
[ 0  1  2  3  4  5  6  7  9 11 12 14 15 16 18 19 20 21 23 24 26 27 28 29
 30 32 33 35 36 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]
Num free chunks:  47
Chunk index:  30
---------------
CHUNK: 358
---------------
Song index:  8
[ 0  1  2  3  4  5  6  7  9 11 12 14 15 16 18 19 20 21 23 24 26 27 28 29
 32 33 35 36 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]
Num free chunks:  46
Chunk index:  19
---------------
CHUNK: 359
---------------
Song index:  8
[ 0  1  2  3  4  5  6  7  9 11 12 14 15 16 18 20 21 23 24 26 27 28 29 32
 33 35 36 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]
Num free chunks:  45
Chunk index:  44
---------------
CHUNK: 360
---------------
Song index:  8
[ 0  1  2  3  4  5  6  7  9 11 12 14 15 16 18 20 21 23 24 26 27 28 29 32
 33 35 36 38 39 40 41 42 43 45 46 47 48 49 50 51 52 53 54 55]
Num free chunks:  44
Chunk index:  28
---------------
CHUNK: 361
---------------
Song index:  8


---------------
CHUNK: 408
---------------
Song index:  9
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 47 48 49 50
 51 52 53 54 56 57 58 59 60 61 62 63 64 65 66]
Num free chunks:  63
Chunk index:  18
---------------
CHUNK: 409
---------------
Song index:  9
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 47 48 49 50 51
 52 53 54 56 57 58 59 60 61 62 63 64 65 66]
Num free chunks:  62
Chunk index:  56
---------------
CHUNK: 410
---------------
Song index:  9
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 47 48 49 50 51
 52 53 54 57 58 59 60 61 62 63 64 65 66]
Num free chunks:  61
Chunk index:  26
---------------
CHUNK: 411
---------------
Song index:  9
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25
 27 28 29 3

---------------
CHUNK: 446
---------------
Song index:  9
[ 6  7  9 10 11 14 15 20 23 28 29 32 34 36 37 39 43 47 48 50 58 60 62 63
 64]
Num free chunks:  25
Chunk index:  47
---------------
CHUNK: 447
---------------
Song index:  9
[ 6  7  9 10 11 14 15 20 23 28 29 32 34 36 37 39 43 48 50 58 60 62 63 64]
Num free chunks:  24
Chunk index:  64
---------------
CHUNK: 448
---------------
Song index:  9
[ 6  7  9 10 11 14 15 20 23 28 29 32 34 36 37 39 43 48 50 58 60 62 63]
Num free chunks:  23
Chunk index:  34
---------------
CHUNK: 449
---------------
Song index:  9
[ 6  7  9 10 11 14 15 20 23 28 29 32 36 37 39 43 48 50 58 60 62 63]
Num free chunks:  22
Chunk index:  43
---------------
CHUNK: 450
---------------
Song index:  9
[ 6  7  9 10 11 14 15 20 23 28 29 32 36 37 39 48 50 58 60 62 63]
Num free chunks:  21
Chunk index:  11
---------------
CHUNK: 451
---------------
Song index:  9
[ 6  7  9 10 14 15 20 23 28 29 32 36 37 39 48 50 58 60 62 63]
Num free chunks:  20
Chunk index:  10
------

In [5]:
train_loader = DataLoader(d, batch_size=32, shuffle=False,
           num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)

In [6]:
import time

start = time.time()
for batch in train_loader:
    print(batch['chunk_index'])

end = time.time()
print('-' * 60)
print('Time elapsed: ', end - start)

Song index:  0
[-] Unloading track None
[+] Loading track 0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32]
Num free chunks:  33
Chunk index:  5
Song index:  0
[ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
Num free chunks:  32
Chunk index:  8
Song index:  0
[ 0  1  2  3  4  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32]
Num free chunks:  31
Chunk index:  31
Song index:  0
[ 0  1  2  3  4  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 32]
Num free chunks:  30
Chunk index:  22
Song index:  0
[ 0  1  2  3  4  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25 26
 27 28 29 30 32]
Num free chunks:  29
Chunk index:  18
Song index:  0
[ 0  1  2  3  4  6  7  9 10 11 12 13 14 15 16 17 19 20 21 23 24 25 26 27
 28 29 30 32]
Num free chunks:  28
Chunk index:  14
Song index:  0
[ 0  1  2  3  4  6  7  9 10 11 12 13 15 16 17 19 20 21 23

Song index:  3
[ 1  3  5  6  7  8 10 11 12 15 16 19 20 22 23 25 26 27 28 30 32 33 34 35
 36 37 38 39 40 42]
Num free chunks:  30
Chunk index:  27
Song index:  3
[ 1  3  5  6  7  8 10 11 12 15 16 19 20 22 23 25 26 28 30 32 33 34 35 36
 37 38 39 40 42]
Num free chunks:  29
Chunk index:  11
Song index:  3
[ 1  3  5  6  7  8 10 12 15 16 19 20 22 23 25 26 28 30 32 33 34 35 36 37
 38 39 40 42]
Num free chunks:  28
Chunk index:  32
Song index:  3
[ 1  3  5  6  7  8 10 12 15 16 19 20 22 23 25 26 28 30 33 34 35 36 37 38
 39 40 42]
Num free chunks:  27
Chunk index:  25
Song index:  3
[ 1  3  5  6  7  8 10 12 15 16 19 20 22 23 26 28 30 33 34 35 36 37 38 39
 40 42]
Num free chunks:  26
Chunk index:  22
Song index:  3
[ 1  3  5  6  7  8 10 12 15 16 19 20 23 26 28 30 33 34 35 36 37 38 39 40
 42]
Num free chunks:  25
Chunk index:  7
Song index:  3
[ 1  3  5  6  8 10 12 15 16 19 20 23 26 28 30 33 34 35 36 37 38 39 40 42]
Num free chunks:  24
Chunk index:  26
Song index:  3
[ 1  3  5  6  8 10 12 15 16 

Song index:  4
[ 4  8 15 21 27 28 34 35 43]
Num free chunks:  9
Chunk index:  28
Song index:  4
[ 4  8 15 21 27 34 35 43]
Num free chunks:  8
Chunk index:  8
Song index:  4
[ 4 15 21 27 34 35 43]
Num free chunks:  7
Chunk index:  21
Song index:  4
[ 4 15 27 34 35 43]
Num free chunks:  6
Chunk index:  4
Song index:  4
[15 27 34 35 43]
Num free chunks:  5
Chunk index:  35
Song index:  4
[15 27 34 43]
Num free chunks:  4
Chunk index:  43
Song index:  4
[15 27 34]
Num free chunks:  3
Chunk index:  34
Song index:  4
[15 27]
Num free chunks:  2
Chunk index:  27
Song index:  4
[15]
Num free chunks:  1
Chunk index:  15
[-] Unloading track 4
Song index:  5
[-] Unloading track None
[+] Loading track 5
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
Num free chunks:  47
Chunk index:  16
Song index:  5
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 3

Song index:  6
[ 0  1  2  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27
 28 29 30 31 33 34 35 37 41]
Num free chunks:  33
Chunk index:  11
Song index:  6
[ 0  1  2  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27 28
 29 30 31 33 34 35 37 41]
Num free chunks:  32
Chunk index:  34
Song index:  6
[ 0  1  2  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27 28
 29 30 31 33 35 37 41]
Num free chunks:  31
Chunk index:  1
Song index:  6
[ 0  2  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27 28 29
 30 31 33 35 37 41]
Num free chunks:  30
Chunk index:  35
Song index:  6
[ 0  2  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27 28 29
 30 31 33 37 41]
Num free chunks:  29
Chunk index:  28
Song index:  6
[ 0  2  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27 29 30
 31 33 37 41]
Num free chunks:  28
Chunk index:  41
Song index:  6
[ 0  2  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27 29 30
 31 33 37]
Num free chunks: 

Song index:  8
[ 2  3  4  8  9 10 12 15 16 17 19 20 24 26 28 31 32 33 34 36 37 38 39 41
 42 43 44 46 48 49 51 53 55]
Num free chunks:  33
Chunk index:  3
Song index:  8
[ 2  4  8  9 10 12 15 16 17 19 20 24 26 28 31 32 33 34 36 37 38 39 41 42
 43 44 46 48 49 51 53 55]
Num free chunks:  32
Chunk index:  34
Song index:  8
[ 2  4  8  9 10 12 15 16 17 19 20 24 26 28 31 32 33 36 37 38 39 41 42 43
 44 46 48 49 51 53 55]
Num free chunks:  31
Chunk index:  20
Song index:  8
[ 2  4  8  9 10 12 15 16 17 19 24 26 28 31 32 33 36 37 38 39 41 42 43 44
 46 48 49 51 53 55]
Num free chunks:  30
Chunk index:  12
Song index:  8
[ 2  4  8  9 10 15 16 17 19 24 26 28 31 32 33 36 37 38 39 41 42 43 44 46
 48 49 51 53 55]
Num free chunks:  29
Chunk index:  36
Song index:  8
[ 2  4  8  9 10 15 16 17 19 24 26 28 31 32 33 37 38 39 41 42 43 44 46 48
 49 51 53 55]
Num free chunks:  28
Chunk index:  37
Song index:  8
[ 2  4  8  9 10 15 16 17 19 24 26 28 31 32 33 38 39 41 42 43 44 46 48 49
 51 53 55]
Num free chunks: 

...

Song index:  95
[ 1  3  6  8 15 17 28]
Num free chunks:  7
Chunk index:  3
Song index:  95
[ 1  6  8 15 17 28]
Num free chunks:  6
Chunk index:  28
Song index:  95
[ 1  6  8 15 17]
Num free chunks:  5
Chunk index:  1
Song index:  95
[ 6  8 15 17]
Num free chunks:  4
Chunk index:  15
Song index:  95
[ 6  8 17]
Num free chunks:  3
Chunk index:  8
Song index:  95
[ 6 17]
Num free chunks:  2
Chunk index:  6
Song index:  95
[17]
Num free chunks:  1
Chunk index:  17
[-] Unloading track 95
Song index:  96
[-] Unloading track None
[+] Loading track 96
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Num free chunks:  48
Chunk index:  40
Song index:  96
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 41 42 43 44 45 46 47]
Num free chunks:  47
Chunk index:  42
tensor([14, 18,  2, 21, 31, 16,  4, 13, 22, 23, 27,  7, 20,  9,  5,

Song index:  97
[ 0  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 23 24 25 26 27
 28 29 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47 50 52]
Num free chunks:  44
Chunk index:  45
Song index:  97
[ 0  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 23 24 25 26 27
 28 29 31 32 34 35 36 37 38 39 40 41 42 43 44 46 47 50 52]
Num free chunks:  43
Chunk index:  39
Song index:  97
[ 0  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 23 24 25 26 27
 28 29 31 32 34 35 36 37 38 40 41 42 43 44 46 47 50 52]
Num free chunks:  42
Chunk index:  0
Song index:  97
[ 2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 23 24 25 26 27 28
 29 31 32 34 35 36 37 38 40 41 42 43 44 46 47 50 52]
Num free chunks:  41
Chunk index:  4
Song index:  97
[ 2  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 23 24 25 26 27 28 29
 31 32 34 35 36 37 38 40 41 42 43 44 46 47 50 52]
Num free chunks:  40
Chunk index:  52
Song index:  97
[ 2  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 23 24 25 26 27 28 2

Song index:  99
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  45  46  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  70  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101]
Num free chunks:  97
Chunk index:  62
Song index:  99
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  45  46  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  63  64  65  66  67  68  70  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  98  99 100 101]
Num free chunks:  96
Chunk index:  100
Song index:  99
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21

Song index:  99
[ 0  1  2  5  6  7  9 10 11 13 14 15 16 17 20 21 23 24 25 29 30 31 32 34
 35 36 39 41 42 46 48 51 52 53 54 55 56 57 58 59 60 61 63 64 66 70 72 73
 74 76 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 97 98 99]
Num free chunks:  70
Chunk index:  79
Song index:  99
[ 0  1  2  5  6  7  9 10 11 13 14 15 16 17 20 21 23 24 25 29 30 31 32 34
 35 36 39 41 42 46 48 51 52 53 54 55 56 57 58 59 60 61 63 64 66 70 72 73
 74 76 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 97 98 99]
Num free chunks:  69
Chunk index:  17
Song index:  99
[ 0  1  2  5  6  7  9 10 11 13 14 15 16 20 21 23 24 25 29 30 31 32 34 35
 36 39 41 42 46 48 51 52 53 54 55 56 57 58 59 60 61 63 64 66 70 72 73 74
 76 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 97 98 99]
Num free chunks:  68
Chunk index:  95
Song index:  99
[ 0  1  2  5  6  7  9 10 11 13 14 15 16 20 21 23 24 25 29 30 31 32 34 35
 36 39 41 42 46 48 51 52 53 54 55 56 57 58 59 60 61 63 64 66 70 72 73 74
 76 80 81 82 83 84 85 86 87 88 89 90 91 92 

Song index:  99
[ 0  2  5  6  9 10 11 13 14 21 24 29 31 32 39 41 46 48 52 54 55 70 76 80
 82 83 87 88 89 92 97 99]
Num free chunks:  32
Chunk index:  48
Song index:  99
[ 0  2  5  6  9 10 11 13 14 21 24 29 31 32 39 41 46 52 54 55 70 76 80 82
 83 87 88 89 92 97 99]
Num free chunks:  31
Chunk index:  52
Song index:  99
[ 0  2  5  6  9 10 11 13 14 21 24 29 31 32 39 41 46 54 55 70 76 80 82 83
 87 88 89 92 97 99]
Num free chunks:  30
Chunk index:  41
Song index:  99
[ 0  2  5  6  9 10 11 13 14 21 24 29 31 32 39 46 54 55 70 76 80 82 83 87
 88 89 92 97 99]
Num free chunks:  29
Chunk index:  29
Song index:  99
[ 0  2  5  6  9 10 11 13 14 21 24 31 32 39 46 54 55 70 76 80 82 83 87 88
 89 92 97 99]
Num free chunks:  28
Chunk index:  83
Song index:  99
[ 0  2  5  6  9 10 11 13 14 21 24 31 32 39 46 54 55 70 76 80 82 87 88 89
 92 97 99]
Num free chunks:  27
Chunk index:  13
Song index:  99
[ 0  2  5  6  9 10 11 14 21 24 31 32 39 46 54 55 70 76 80 82 87 88 89 92
 97 99]
Num free chunks:  26
Chunk ind